<a href="https://colab.research.google.com/github/Shujaat123/ECM-LSE/blob/master/ECM_LSE_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **ECM-LSE: Extracellular Matrix Proteins Prediction Using Latent Space Encoding of Composition of k-Spaced Amino Acid Pairs**


This code provide python implementation of ECM-LSE algorithm.

# Loading Useful packages

In [0]:
## Load useful packages
import sys, os, re, gc
from scipy.io import savemat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils.np_utils import to_categorical
from collections import Counter
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization, Dropout
from keras import optimizers
from keras.utils.np_utils import to_categorical
from keras import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score, matthews_corrcoef, balanced_accuracy_score, precision_recall_fscore_support
from keras.models import load_model
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from random import sample

# Feature-Extraction

The CKSAAP feature encoding calculates the frequency of amino acid pairs separated by any k residues. The CKSAAP encoding scheme reflects the amino acid pair information in small and large range with in the peptides depending upon the value of k(gap). The encoding scheme is utilized from iFeature web server, using the following download link: (https://github.com/Superzchen/iFeature).


In [0]:
## Define CKSAAP feature-extraction function
def minSequenceLength(fastas):
	minLen = 10000
	for i in fastas:
		if minLen > len(i[1]):
			minLen = len(i[1])
	return minLen

def CKSAAP(fastas, gap=5, **kw):
	if gap < 0:
		print('Error: the gap should be equal or greater than zero' + '\n\n')
		return 0

	if minSequenceLength(fastas) < gap+2:
		print('Error: all the sequence length should be larger than the (gap value) + 2 = ' + str(gap+2) + '\n\n')
		return 0

	AA = 'ACDEFGHIKLMNPQRSTVWY'
	encodings = []
	aaPairs = []
	for aa1 in AA:
		for aa2 in AA:
			aaPairs.append(aa1 + aa2)
	header = ['#']
	for g in range(gap+1):
		for aa in aaPairs:
			header.append(aa + '.gap' + str(g))
	encodings.append(header)
	for i in fastas:
		name, sequence = i[0], i[1]
		code = [name]
		for g in range(gap+1):
			myDict = {}
			for pair in aaPairs:
				myDict[pair] = 0
			sum = 0
			for index1 in range(len(sequence)):
				index2 = index1 + g + 1
				if index1 < len(sequence) and index2 < len(sequence) and sequence[index1] in AA and sequence[index2] in AA:
					myDict[sequence[index1] + sequence[index2]] = myDict[sequence[index1] + sequence[index2]] + 1
					sum = sum + 1
			for pair in aaPairs:
				code.append(myDict[pair] / sum)
		encodings.append(code)
	return encodings


# Designing an Auto-Encoder-based classifier model

In [0]:
## Designing an Auto-Encoder-Classifier model
def AFP_LSE_Final_Model(LV=5):
    # Encoder Network
    enc_input = Input(shape=(3600,), name='enc_input')
    enc_l1 = Dense(50, activation='relu', name='encoder_layer1')(enc_input)
    enc_l1 = BatchNormalization()(enc_l1)
    enc_l1 = Dropout(rate=0.7)(enc_l1)

    enc_l2 = Dense(25, activation='relu', name='encoder_layer2')(enc_l1)
    enc_l2 = BatchNormalization()(enc_l2)
    enc_l2 = Dropout(rate=0.3)(enc_l2)

    enc_l3 = Dense(25, activation='relu', name='encoder_layer3')(enc_l2)
    enc_l3 = BatchNormalization()(enc_l3)
    enc_l3 = Dropout(rate=0.3)(enc_l3)

    enc_l4 = Dense(10, activation='relu', name='encoder_layer4')(enc_l3)
    enc_l4 = BatchNormalization()(enc_l4)
    enc_l4 = Dropout(rate=0.3)(enc_l4)

    encoder_output = Dense(LV, activation='sigmoid', name='encoder_output')(enc_l4)

    # Classifier Network
    class_l1 = Dense(10, activation='relu', name='class_layer1')(encoder_output)
    class_l2 = Dense(10, activation='relu', name='class_layer2')(class_l1)
    class_l3 = Dense(10, activation='relu', name='class_layer3')(class_l2)
    class_output = Dense(2, activation='softmax', name='class_output')(class_l3)

    # Decoder Network
    dec_l1 = Dense(10, activation='relu', name='decoder_layer1')(encoder_output)
    dec_l1 = BatchNormalization()(dec_l1)
    dec_l1 = Dropout(0.7)(dec_l1)

    dec_l2 = Dense(25, activation='relu', name='decoder_layer2')(dec_l1)
    dec_l2 = BatchNormalization()(dec_l2)
    dec_l2 = Dropout(0.3)(dec_l2)

    dec_l3 = Dense(25, activation='relu', name='decoder_layer3')(dec_l2)
    dec_l3 = BatchNormalization()(dec_l3)
    dec_l3 = Dropout(0.3)(dec_l3)

    dec_l4 = Dense(50, activation='relu', name='decoder_layer4')(dec_l3)
    dec_l4 = BatchNormalization()(dec_l4)
    dec_l4 = Dropout(0.3)(dec_l4)

    decoder_output = Dense(3600, activation='sigmoid', name='decoder_output')(dec_l4)

    model = Model(inputs=[enc_input], outputs=[class_output, decoder_output])

    # Compiling model
    model.compile(optimizer='rmsprop',
                  loss={'class_output': 'binary_crossentropy', 'decoder_output': 'mean_squared_error'},
                  loss_weights={'class_output': 1., 'decoder_output': 0.01},
                  metrics=[metrics.categorical_accuracy])
    # Here I used rmsprops optimizer with default values, two objective functions are optimized
    # using  weight factors [1 for classifier and 0.1 for decoder loss]
    return model



## Define performance measures

In [0]:
## Define performance measures
def yoden_index(y, y_pred):
  tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
  j = (tp/(tp+fn)) + (tn/(tn+fp)) - 1
  return j

def pmeasure(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1score = 2 * tp / (2 * tp + fp + fn)
    return ({'Sensitivity': sensitivity, 'Specificity': specificity, 'F1-Score': f1score})

def Show_Statistics(msg,Stats):
  print(msg.upper())
  print(70*'-')
  print('Accuracy:',Stats[0])
  print('Sensitivity:',Stats[1])
  print('Specificity:',Stats[2])
  print('F1-Score:',Stats[3])
  print('MCC:',Stats[4])
  print('Balance Accuracy:',Stats[5])
  print('Youden-Index:',Stats[6])
  print('Reconstruction MSE:',Stats[7])
  print(70*'-')

## Loading and pre-processing ECM prediction dataset

In [0]:
## Loading and pre-processing AFP prediction dataset
data_path = 'https://raw.githubusercontent.com/Shujaat123/ECM-LSE/master/Dataset.csv'
dataset = pd.read_csv(data_path, index_col=None)

seq=[]

for index, row in dataset.iterrows():
  array = [row['Class'], row['Sequence']]
  name, sequence = array[0].split()[0], re.sub('[^ARNDCQEGHILKMFPSTWYV-]', '-', ''.join(array[1:]).upper())
  seq.append([name, sequence])


cksaapfea = []
for i in seq:
  temp= CKSAAP([i], gap=8)
  cksaapfea.append(temp)

dt = []
for i in range(len(cksaapfea)):
  temp = cksaapfea[i][1][1:]
  dt.append(temp)

dtn = np.array(dt)
from keras.utils.np_utils import to_categorical
y = dataset['Class']
y[y=='AFP']=1
y[y=='NON-AFP']=0
y = to_categorical(y)
LVs = [5]
num_Trails = 20

  ## Perform Monte-Carlos Simulations for [num_Trials]\# of independent Trials


In [0]:
  ## Perform Monte-Carlos Simulations for [num_Trials]# of independent Trials
  Stats =[]

  for loop_ind in range(0,num_Trails):
      # from random import sample

      plist = list(np.asarray(np.where(y[:,1]==1)).flatten())
      nlist = list(np.asarray(np.where(y[:,1]==0)).flatten())

      ## train
      p_train = sample(plist, 270)
      n_train = sample(nlist, 270)
      train_list = p_train + n_train
      X_train = dtn[p_train + n_train]
      y_train = y[p_train + n_train]

      ## valid
      p_val_list = set(plist) - set(p_train)
      n_val_list = set(nlist) - set(n_train)
      p_val = sample(p_val_list, 30)
      n_val = sample(n_val_list, 30)
      X_val = dtn[list(p_val)+list(n_val)]
      y_val = y[list(p_val)+list(n_val)]
      val_list = list(p_val) + list(n_val)

      ## test
      dev_list = train_list + val_list
      test_list = set(list(np.where(y)[0])) - (set(dev_list))
      X_test = dtn[list(test_list)]
      y_test = y[list(test_list)]

      model = AFP_LSE_Final_Model(LV=LVs[0])

      es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=300)

      checkpoint = ModelCheckpoint('models\\model-best.h5',
                                    verbose=0, monitor='val_loss',save_best_only=True, mode='auto')


      history = model.fit({'enc_input': X_train},
                          {'class_output': y_train, 'decoder_output': X_train},
                          validation_data = ({'enc_input': X_val},
                          {'class_output': y_val, 'decoder_output': X_val}),
                          epochs=700, batch_size=100, callbacks=[checkpoint, es], verbose=0)

      del model  # deletes the existing model
      model = load_model('models\\model-best.h5')

      y_train_pred, X_train_pred = model.predict(X_train,batch_size=540, verbose=0)
      y_train_pred = to_categorical(y_train_pred.argmax(axis=1))
      MSE_X_train_pred = (np.square(X_train_pred - X_train)).mean(axis=1)

      y_val_pred, X_val_pred = model.predict(X_val,batch_size=60, verbose=0)
      y_val_pred = to_categorical(y_val_pred.argmax(axis=1))
      MSE_X_val_pred = (np.square(X_val_pred - X_val)).mean(axis=1)

      y_test_pred, X_test_pred = model.predict(X_test,batch_size=9074, verbose=0)
      y_test_pred = to_categorical(y_test_pred.argmax(axis=1))
      MSE_X_test_pred = (np.square(X_test_pred - X_test)).mean(axis=1)

     
      ## Training Measures
      tr_acc = accuracy_score(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))
      tr_sen = pmeasure(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))['Sensitivity']
      tr_spe = pmeasure(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))['Specificity']
      tr_f1 = pmeasure(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))['F1-Score']
      tr_mcc = matthews_corrcoef(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))
      tr_bacc = balanced_accuracy_score(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))
      tr_yi = yoden_index(y_train.argmax(axis=1), y_train_pred.argmax(axis=1))

      ## Validation Measures
      v_acc = accuracy_score(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))
      v_sen = pmeasure(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))['Sensitivity']
      v_spe = pmeasure(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))['Specificity']
      v_f1 = pmeasure(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))['F1-Score']
      v_mcc = matthews_corrcoef(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))
      v_bacc = balanced_accuracy_score(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))
      v_yi = yoden_index(y_val.argmax(axis=1), y_val_pred.argmax(axis=1))

      ## Test Measures
      t_acc = accuracy_score(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))
      t_sen = pmeasure(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))['Sensitivity']
      t_spe = pmeasure(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))['Specificity']
      t_f1 = pmeasure(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))['F1-Score']
      t_mcc = matthews_corrcoef(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))
      t_bacc = balanced_accuracy_score(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))
      t_yi = yoden_index(y_test.argmax(axis=1), y_test_pred.argmax(axis=1))


      Stats.append([tr_acc, tr_sen, tr_spe, tr_f1, tr_mcc, tr_bacc, tr_yi, -10*np.log10(MSE_X_train_pred.mean()),
                    v_acc, v_sen, v_spe, v_f1, v_mcc, v_bacc, v_yi, -10*np.log10(MSE_X_val_pred.mean()),
                    t_acc, t_sen, t_spe, t_f1, t_mcc, t_bacc, t_yi, -10*np.log10(MSE_X_test_pred.mean())])
      print('LV=',LVs[0],'Trial:',loop_ind)

  Statistics = np.asarray(Stats)


## Show Classification/Reconstruction Statistics

In [0]:
## Show Classification/Reconstruction Statistics
Show_Statistics('Training Results (MEAN)',Statistics.mean(axis=0)[0:8])
Show_Statistics('Validation Results (MEAN)',Statistics.mean(axis=0)[8:16])
Show_Statistics('Test Results (MEAN)',Statistics.mean(axis=0)[16:24])